In [4]:
import panel as pn
import panel.widgets as pnw
import pandas as pd
import numpy as np
from panel.interact import interact
from IPython.display import Image
from IPython.core.display import HTML
import param
from pathlib import Path 
import hvplot.pandas
from holoviews import opts
pn.extension()

In [2]:
#Dictionaries for plot storing

# #Dictionary storing logos
# logo_dict = [
#     {'Crypto':'BTC','Logo':Image(filename='Plots/btc_logo.png')},
#     {'Crypto':'ETH','Logo':Image(filename='Plots/eth_logo.png')},
#     {'Crypto':'Doge', 'Logo':Image(filename='Plots/doge_logo.png')}
# ]

# #Each dictionary stores a different type of plot
# heat_dict_d = [
#     {'Crypto':'BTC','Heat_d':Image(filename='Plots/btc_heat_d.png')},
#     {'Crypto':'ETH','Heat_d':Image(filename='Plots/eth_heat_d.png')},
#     {'Crypto':'Doge', 'Heat_d':Image(filename='Plots/doge_heat_d.png')}
# ]



In [5]:
mortality_file_path = Path('Resources/Mortality.csv', header = 0)
mortality_data = pd.read_csv(mortality_file_path)
mortality_data = mortality_data.drop(['Split', 'SplitSex', 'Forecast'], axis = 1)
mortality_data_country_indexed = mortality_data.set_index(mortality_data['CountryCode'])
mortality_data_country_indexed.drop(columns=['CountryCode'], inplace=True)
# mortality_data_country_indexed.drop(1, inplace = True)
mortality_data_country_indexed.columns = ['Year', 'Week', 'Sex',
                                          'Deaths: 0-14 yrs',
                                          'Deaths: 15-64 yrs',
                                          'Deaths: 65-74 yrs',
                                          'Deaths: 75-84 yrs',
                                          'Deaths: 85+ yrs',
                                          'Deaths: Total',
                                          'Death Rate: 0-14 yrs',
                                          'Death Rate: 15-64 yrs',
                                          'Death Rate: 65-74 yrs',
                                          'Death Rate: 75-84 yrs',
                                          'Death Rate: 85+ yrs',
                                          'Death Rate: Total']
# mortality_data_country_indexed.head()

In [6]:
country_codes = mortality_data['CountryCode'].unique()
# print(country_codes)

In [7]:
country_dict = dict(zip(country_codes, ['Australia', 'Austria', 'Belgium', 'Bulgaria', 'Canada', 'Switzerland', 'Chile', 'Czech Republic', 'Germany', 'Denmark',
                                        'Spain', 'Estonia', 'Finland', 'France', 'England and Wales', 'Northern Ireland', 'Scotland', 'Greece', 
                                        'Croatia', 'Hungary', 'Iceland', 'Israel', 'Italy', 'South Korea', 'Lithuania', 'Luxembourg', 'Latvia', 
                                        'Netherlands', 'Norway', 'New Zealand', 'Poland', 'Portugal', 'Russia', 'Slovakia', 'Slovenia', 'Sweden', 'Taiwan', 'USA']))
# print(country_dict)

In [8]:
# Turning dictionary into a dataframe, should help organize dropdown selector with param
dash_df = pd.DataFrame.from_dict(country_dict, orient='index')
dash_df.rename_axis('codes', inplace=True)
dash_df.rename(columns={0:'country'}, inplace=True)
# dash_df

In [ ]:
#Dashboard construction

# Class containing dropdown menu for cryptos, relevant plots and dataframes
class MortalityDashboard(param.Parameterized):
    
    #Crypto dropdown selector
    Country = param.ObjectSelector(default='Australia', objects=list(dash_df.country.unique()))
    
#     def rel_data(self):
#         crypto_data = dash_df[(dash_df.Crypto==self.Crypto)].copy()
#         return crypto_data

    

In [ ]:
# Optionally creating a multiselect dropdown type menu for contries, maybe better
multi_select = pn.widgets.MultiSelect(name='Countries', value=['Australia'],
    options= country_dict, size=6)
multi_select

In [ ]:
covid_logo_dict = pn.panel('Resources/covlogos.png', width=200, align='start')
covid_logo_dict

In [ ]:
# Dynamic bar graph from slider and dropdown
years_df = mortality_data_country_indexed[mortality_data_country_indexed.Year.between(2015, 2020)]
# years_df
# bar_table = years_df["Deaths: 0-14 yrs"]
# bar_table
years_df.hvplot(y=['Deaths: 0-14 yrs', 'Deaths: 15-64 yrs'], x="Week", groupby=["CountryCode", "Sex"])

In [ ]:
# Creating a year slider for selecting specific years worth of data, could be modified to focus on specific weeks/months
year_slider = pn.widgets.IntRangeSlider(name='Years Slider', width=300, start=2015, end=2020, value=(2015, 2020), value_throttled=(2015, 2020))
year_slider

In [ ]:
# Dynamic markup title for slider
@pn.depends(year_slider.param.value_throttled)
def year_range(year_slider):
    return '### Yearly Data Between {start} —  {end}'.format(start=year_slider[0], end=year_slider[1])

In [ ]:
pn.Row(year_range)

In [ ]:
# Bar chart construction
@pn.depends(year_slider.param.value_throttled)
def plot_bar(year_slider):
    years_df = mortality_data_country_indexed[mortality_data_country_indexed.Year.between(year_slider[0], year_slider[1])]
    bar_table = years_df["Deaths: 0-14 yrs"].reset_index()
    return bar_table[:10].sort_values("Deaths: 0-14 yrs").hvplot(invert=False, legend="bottom_right", height=400)

pn.Row(plot_bar)

In [ ]:
CD = MortalityDashboard()

In [ ]:
# Dashboard elements
dash_title = "# Country Mortality Analysis"

# Dashboard description
dash_desc = "Our project aims to visualize different metrics of COVID-19, specifically focusing on excess death"
text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum."

# Dashboard logo
covid_logo_dict = pn.panel('Resources/covlogos.png', width=250, align='center')

# Header
header_box = pn.WidgetBox(dash_title, 
                          dash_desc, 
                          covid_logo_dict, 
#                           CD.param, 
                          multi_select,
                          year_slider, 
                          pn.Row(year_range),
                          plot_bar,
                          pn.layout.Spacer(margin=125), 
                          text, 
                          width=1000, 
                          height=1000, 
                          align="center"
                         )

#Placing plots in tabs

# # Basic tab construction, with tab name and respective plot
# heat_tabs = pn.Tabs(
#     ("Daily", CD.heat_plot_d),
#     ("Weekly", CD.heat_plot_w),
#     ("Monthly", CD.heat_plot_m),
#     ("Yearly", CD.heat_plot_y)
# )

# # Rows containing two plots, respective tabs will have name of analysis and plots

# pol_row = pn.Row(
#     CD.pol_plot, CD.vpol_plot
# )

# comp_row = pn.Row(
#     CD.vcomp_plot, CD.vcompr_plot
# )

# lstm_rnn_row = pn.Row(
#     CD.lstm_plot, CD.rnn_plot
# )

# cm_tabs = pn.Tabs(
#     ("Polarity", pol_row),
#     ("Compound", comp_row),
#     ("LSTM-RNN", lstm_rnn_row)
# )

#Dashboard construction
dashboard = pn.Column(
    header_box
#     dash_title,
#     dash_desc,
#     mortality_row,
#     "## Heatmap",
#     heat_tabs,
#     "## Line Plot",
#     line_tabs,
#     "## Scatter Plot",
#     CD.scatter_plot,
#     "## Histograms",
#     hist_tabs,
#     cm_tabs
)

In [ ]:
dashboard.embed()